___
___
___
# CLASSIFICATION
___
___
___

In [64]:
import pandas as pd

df = pd.read_csv('../dataset.csv')

# on va faire du LLM avec python pandas scikit-learn
# voici mon contexte : j'ai récupérer des évènements (artisitques, etc.) sur internet. j'ai un csv avec comme features : url	title	description. De plus j'ai trois target : cat1	cat2	cat3
# je voudrai faire de la catégorisation à partir de ces csv. Je ne suis pas sûr que l'URL soit totalement pertinent. Peut être faut-il récupérer le nom de domaine.

___
### 1. Préparation des données
___
tout d'abord, vous devez charger les données à partir du fichier CSV en utilisant Pandas. Ensuite, vous pouvez extraire le nom de domaine à partir de l'URL en utilisant la bibliothèque tldextract. Vous pouvez ensuite ajouter une nouvelle colonne au DataFrame Pandas contenant le nom de domaine.

In [65]:
def extract_domain(url):
    url = url[url.find('//')+2:]
    url = url[:url.find('/')]
    return url

df['domain'] = df['url'].apply(extract_domain)
df[['domain', 'url']].head(1)

,domain,url
0,www.tourisme-cambresis.fr,https://www.tourisme-cambresis.fr/1-les-templi...


### 1.1 les catégories cat2 et cat3 possèdent des NaN
création de df spécifique pour chaque target

In [66]:
df1 = df.copy()
df2 = df.dropna(subset=['cat2'])
df3 = df.dropna(subset=['cat3'])

___
### 2. Prétraitement des données
___
vous devez ensuite prétraiter les données textuelles dans les colonnes title et description. Vous pouvez par exemple supprimer les ponctuations, les chiffres et les caractères spéciaux, convertir les textes en minuscules, supprimer les stop-words et effectuer une stemming ou une lemmatization. Vous pouvez utiliser la bibliothèque nltk pour effectuer ces opérations.

In [67]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Vérifier que la valeur d'entrée est une chaîne de caractères
    if not isinstance(text, str):
        return ''
    
    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Supprimer les ponctuations, les chiffres et les caractères spéciaux
    tokens = [token for token in tokens if token.isalpha()]

    # Convertir en minuscules
    tokens = [token.lower() for token in tokens]

    # Supprimer les stop-words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Reconstruction du texte prétraité
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

display(df1.head(1))
df1['title'] = df1['title'].apply(preprocess_text)
df1['description'] = df1['description'].apply(preprocess_text)
display(df1.head(1))

display(df2.head(1))
df2['title'] = df2['title'].apply(preprocess_text)
df2['description'] = df2['description'].apply(preprocess_text)
display(df2.head(1))

display(df3.head(1))
df3['title'] = df3['title'].apply(preprocess_text)
df3['description'] = df3['description'].apply(preprocess_text)
display(df3.head(1))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/utilisateur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/utilisateur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,url,title,description,cat1,cat2,cat3,domain
0,0,https://www.tourisme-cambresis.fr/1-les-templi...,"Aventure-jeu : ""Les Templiers du coffre d'or""",Le jeu aventure « Les templiers du coffre d’or...,Jeu,Famille,Détente,www.tourisme-cambresis.fr


,Unnamed: 0,url,title,description,cat1,cat2,cat3,domain
0,0,https://www.tourisme-cambresis.fr/1-les-templi...,le templier du coffr,le jeu aventur le templier du coffr créé par l...,Jeu,Famille,Détente,www.tourisme-cambresis.fr


,Unnamed: 0,url,title,description,cat1,cat2,cat3,domain
0,0,https://www.tourisme-cambresis.fr/1-les-templi...,"Aventure-jeu : ""Les Templiers du coffre d'or""",Le jeu aventure « Les templiers du coffre d’or...,Jeu,Famille,Détente,www.tourisme-cambresis.fr


/tmp/ipykernel_58017/1738319832.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['title'] = df2['title'].apply(preprocess_text)
/tmp/ipykernel_58017/1738319832.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['description'] = df2['description'].apply(preprocess_text)


,Unnamed: 0,url,title,description,cat1,cat2,cat3,domain
0,0,https://www.tourisme-cambresis.fr/1-les-templi...,le templier du coffr,le jeu aventur le templier du coffr créé par l...,Jeu,Famille,Détente,www.tourisme-cambresis.fr


,Unnamed: 0,url,title,description,cat1,cat2,cat3,domain
0,0,https://www.tourisme-cambresis.fr/1-les-templi...,"Aventure-jeu : ""Les Templiers du coffre d'or""",Le jeu aventure « Les templiers du coffre d’or...,Jeu,Famille,Détente,www.tourisme-cambresis.fr


/tmp/ipykernel_58017/1738319832.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['title'] = df3['title'].apply(preprocess_text)
/tmp/ipykernel_58017/1738319832.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['description'] = df3['description'].apply(preprocess_text)


,Unnamed: 0,url,title,description,cat1,cat2,cat3,domain
0,0,https://www.tourisme-cambresis.fr/1-les-templi...,le templier du coffr,le jeu aventur le templier du coffr créé par l...,Jeu,Famille,Détente,www.tourisme-cambresis.fr


___
### 3.Vectorisation des données
___
vous devez ensuite convertir les données textuelles prétraitées en vecteurs de caractéristiques en utilisant par exemple TfidfVectorizer de Scikit-learn. Vous pouvez concaténer les vecteurs obtenus à partir des colonnes title, description et éventuellement domain.

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

vectorizer = TfidfVectorizer()
X1_title = vectorizer.fit_transform(df1['title'])
X1_description = vectorizer.transform(df1['description'])
X1_domain = vectorizer.transform(df1['domain'])

# Concaténer les vecteurs
# from sklearn.datasets import load_files
X1 = scipy.sparse.hstack([X1_title, X1_description, X1_domain])

vectorizer = TfidfVectorizer()
X2_title = vectorizer.fit_transform(df2['title'])
X2_description = vectorizer.transform(df2['description'])
X2_domain = vectorizer.transform(df2['domain'])

# Concaténer les vecteurs
# from sklearn.datasets import load_files
X2 = scipy.sparse.hstack([X2_title, X2_description, X2_domain])

vectorizer = TfidfVectorizer()
X3_title = vectorizer.fit_transform(df3['title'])
X3_description = vectorizer.transform(df3['description'])
X3_domain = vectorizer.transform(df3['domain'])

# Concaténer les vecteurs
# from sklearn.datasets import load_files
X3 = scipy.sparse.hstack([X3_title, X3_description, X3_domain])


___
### 3. Entraînement du modèle
___
vous pouvez ensuite entraîner un modèle de catégorisation à partir des vecteurs de caractéristiques et des colonnes cat1, cat2 et cat3 en utilisant par exemple LogisticRegression ou RandomForestClassifier de Scikit-learn.

In [69]:
from sklearn.linear_model import LogisticRegression

y1 = df1['cat1']
y2 = df2['cat2']
y3 = df3['cat3']

model1 = LogisticRegression()
model1.fit(X1, y1)

model2 = LogisticRegression()
model2.fit(X2, y2)

model3 = LogisticRegression()
model3.fit(X3, y3)


LogisticRegression()